# pandas exercises

We'll use the sample dataset from the Social Secury Administration on baby names:
https://www.ssa.gov/oact/babynames/limits.html

Download the "National" version and unzip it.  There will be one file for each year.

This example follows the _Python for Data Analysis_ book closely

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'pandas'

Let's start by reading in just a single dataset, for the first year available (1880).  We give the names of the columns here.  The index will just be the line / record number in the file (not really important for us)

In [2]:
names_1880 = pd.read_csv('babynames/yob1880.txt', names=["name", "sex", "births"])
names_1880

NameError: name 'pd' is not defined

## number of births

Now we want to start doing some analysis.  `.groupby()` creates a new type of container that groups items.  Here, we'll group by sex.

See this:
http://pandas.pydata.org/pandas-docs/stable/groupby.html
https://community.modeanalytics.com/python/tutorial/pandas-groupby-and-python-lambda-functions/

for a nice explanation of this method.

In [3]:
names_1880.groupby("sex")["births"].sum()

NameError: name 'names_1880' is not defined

Note that we can use a column name as a parameter of the object, so we can equivalently do this as:

In [4]:
names_1880.groupby("sex").births.sum()

NameError: name 'names_1880' is not defined

On its own, you can't print a `GroupBy` object.  You need to do a transformation or filter on it to get back a `Series` or `DataFrame`.

In [5]:
gb = names_1880.groupby("sex")
gb.count()

NameError: name 'names_1880' is not defined

As we saw in the intro, we can propagate a scalar to all indices in the `DataFrame`.  We'll use this to add a new column that represents the year:

In [6]:
names_1880["year"] = 1880
names_1880

NameError: name 'names_1880' is not defined

## all data sets

Now we can read in all of the datasets.  We will read them in one-by-one and add the "year" column to them.  We'll keep a list of all the `DataFrame`s and then we'll use a pandas method to transform this list of `DataFrame`s into a single `DataFrame`

In [7]:
years = range(1880, 2016)

pieces = []
columns = ["name", "sex", "births"]

for yr in years:
    frame = pd.read_csv("babynames/yob{}.txt".format(yr), names=columns)
    
    # this creates a new column and gives all rows the same value -- the year
    frame["year"] = yr
    pieces.append(frame)

# create a single dataframe through concatenation -- ignore_index means that
# we don't care about the original line numbers
names = pd.concat(pieces, ignore_index=True)

NameError: name 'pd' is not defined

In [8]:
names

NameError: name 'names' is not defined

## pivot tables

a _pivot table_ creates a new dataframe from our original one, usually summarizing the data in a new way.  In particular, with a pivot table, we can create a new index and columns, with the data in the `DataFrame` reduced via some operation across another column.

Here, the column that we are going to aggregate is "births", and the function will will use for the aggregating is `sum` (to sum over the names).

In [9]:
names.head()

NameError: name 'names' is not defined

In [10]:
total_births = names.pivot_table("births", index="year", columns="sex", aggfunc=sum)

NameError: name 'names' is not defined

In [11]:
total_births

NameError: name 'total_births' is not defined

In [12]:
total_births.plot()

NameError: name 'total_births' is not defined

## `.apply()` and new columns

Now we'll use `.groupby()` again to allow use to derive a new column -- the fraction of the births with the name specified by the index.  

`apply()` uses what Pandas calls the _split-apply-combine_ rules, e.g., it splits the data into groups, applies your function, and then intelligently concatenates it back together.  Here, the result will be a `DataFrame` with the original information and this new column.

In [13]:
def add_prop(group):
    group["prop"] = group["births"]/group["births"].sum()
    return group

names = names.groupby(["year", "sex"]).apply(add_prop)

NameError: name 'names' is not defined

In [14]:
names

NameError: name 'names' is not defined

## Q1: Sanity check

Verify that within each of the groups we just used above that the "prop" column sums to 1 (it should be close, to roundoff).  The `np.allclose()` function might be useful here.

Now we will group by "year" and "sex" and then sort by births to get just the top N most popular names.

Notice that we simply append any additional arguments to our apply function after the function name (this is an example of the `**kwargs` behavior in python

In [15]:
def get_top(group, N=1000):
    return group.sort_values(by="births", ascending=False)[:N]

grouped = names.groupby(["year", "sex"])
top = grouped.apply(get_top, 1000)

NameError: name 'names' is not defined

In [16]:
top

NameError: name 'top' is not defined

## Q2: split by sex

create two new dataframes, one `boys` with just those in `top` that are "M" and one `girls` with those in `top` who are "F"

In [17]:
boys

NameError: name 'boys' is not defined

## More analysis

How many times does each name appear, by year?  This dataframe is the total number of births by year and name

In [18]:
total_births = top.pivot_table("births", index="year", columns="name", aggfunc=sum)

NameError: name 'top' is not defined

In [19]:
total_births

NameError: name 'total_births' is not defined

In [20]:
total_births["John"]

NameError: name 'total_births' is not defined

We can see that this matches what we had in our previous data frame

In [21]:
top[top.name=="John"]

NameError: name 'top' is not defined

Which names are the most popular of all time?

In [22]:
a = total_births.sum()
a.sort_values(ascending=False)

NameError: name 'total_births' is not defined

Let's plot naming trends, for a few of the most popular names

In [23]:
subset = total_births[["Michael", "Jennifer"]]
subset.plot()

NameError: name 'total_births' is not defined

# Exercises

## Q3: unique names

how many unique names appear in our top 1000 list?  Use the `.unique()` method on the "name" `Series` to get a an array (it will actually be a NumPy `ndarray` of objects)

## Q4: gender neutral names

What are all the names that appear for both boys and girls?

## Q5: name diversity

We want to make a plot of how many names it takes to reach 50% of the births in a given year.  Let's start with the boys names:

In [24]:
boys = top[top.sex == "M"]

NameError: name 'top' is not defined

Now, to understand the process, let's work just on a particular year, 2015

In [25]:
b15 = boys[boys.year == 2015]

NameError: name 'boys' is not defined

In [26]:
b15.info

NameError: name 'b15' is not defined

We'll use the `cumsum()` property to do a cumulative sum over the `prop` column we added previously

In [27]:
prop_cumsum = b15.sort_values(by="prop", ascending=False)["prop"].cumsum()

NameError: name 'b15' is not defined

and now we can use `searchsorted` to return the position where "if the corresponding elements in v were inserted before the indices, the order of self would be preserved." (see http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.searchsorted.html)

In [28]:
prop_cumsum.searchsorted(0.5)

NameError: name 'prop_cumsum' is not defined

Since we start counting at 0, we add 1 to this to get the number of names needed in this year to reach 50% of births.

Now generalize this to a function that you can use `apply()` on and make a plot of the number of names to reach 50% by year